CONTENT WARNING: explicit and multiple references to suicide attemps and suicidal ideation

The following is an analysis regarding attempted suicides and suicidal thoughts. Questions around these topics will be directly referenced and mentioned explicitly. While their potential correlations with other parameters will be analysed and discussed at length, suicide attempts and suicidal ideation as concepts will not be discussed. Deaths by suicide will not be referenced or discussed at all.

Please only continue if it comfortable for you to do so.

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.cross_validation import train_test_split
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle

/anaconda3/envs/condasc/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


2012 through 2016 results ( https://www.datafiles.samhsa.gov/study/national-survey-drug-use-and-health-nsduh-2002-2016-nid18454 )

In [2]:
df = pd.DataFrame.from_csv('NSDUH_2016_Tab.tsv',sep='\t')

/anaconda3/envs/condasc/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
/anaconda3/envs/condasc/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (2506) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [37]:
def prepdf(df):
    dfc = df.copy()
    dfc = dfc[(dfc.MHSUITHK==0) | (dfc.MHSUITHK==1)]
    dfc.drop(columns=['SUICTHNK','FILEDATE'],inplace=True)
    
    dfc.HTINCHE2 = np.floor((dfc.HTINCHE2-55)/5)
    dfc.WTPOUND2 = np.floor((dfc.WTPOUND2-75)/55)
    dfc.BMI2 = np.floor((dfc.BMI2-9.3)/14.83)
    
    tmp_array = np.arange(80,100,1)
    rep_array = np.append(tmp_array,tmp_array+100)
    rep_array = np.append(rep_array,tmp_array+900)
    rep_array = np.append(rep_array,tmp_array+9900)
    
    dfc = dfc.replace(rep_array,np.nan)
    
    return dfc

In [38]:
dftest = prepdf(df)
dftest

,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,POVERTY3,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP
QUESTID2,,,,,,,,,,,,,,,,,,,,,
11635143,1,NaN,NaN,16.0,NaN,NaN,4.0,NaN,NaN,NaN,...,3.0,2.0,2.0,3,3,2,2,819.434247,40037,2
36845143,1,NaN,NaN,15.0,NaN,NaN,1.0,7.0,NaN,2.0,...,3.0,1.0,2.0,2,2,2,2,280.624352,40013,2
35755143,1,NaN,NaN,26.0,NaN,NaN,1.0,7.0,NaN,4.0,...,2.0,2.0,2.0,1,1,2,2,10133.833583,40036,2
92675143,1,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,...,3.0,2.0,2.0,1,1,2,2,24815.892373,40009,2
59596143,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,2.0,2,2,2,2,5546.690814,40016,1
41106143,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,2.0,2,2,2,2,9539.366625,40002,1
96416143,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,2.0,2.0,2,2,2,2,4144.946988,40021,2
73716143,1,NaN,NaN,15.0,NaN,NaN,1.0,30.0,NaN,4.0,...,1.0,2.0,2.0,2,2,2,2,2940.093267,40009,2
76226143,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,2.0,3,3,2,2,1781.771853,40025,2


In [ ]:
def 